https://huggingface.co/psyche/KoT5-summarization

In [ ]:
!pip install transformers

In [1]:
import pandas as pd

import torch
from tqdm import tqdm

In [ ]:
song_df = pd.read_csv('가사_스테디셀러.csv')
# song_df = pd.read_csv('가사.csv')

In [ ]:
song_df

,Song_ID,Genre,Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630,90105,랩/힙합,This is the tihs that y'all ain't ready for Th...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...


In [ ]:
import re

# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\\t', '', sentence)
        sentence = re.sub('\\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

In [ ]:
song_df['Lyrics'] = preprocess(song_df['Lyrics'])

In [ ]:
song_df

,Song_ID,Genre,Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630,90105,랩/힙합,지누션함께면 너도 새로 태어난 힙합 빠삐용 지누...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...


In [ ]:
model_name = 'psyche/KoT5-summarization'

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# device=0 을 지정해주면, GPU로 돌리게 된다.
pipe = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

In [ ]:
result = pipe("맛있는 걸 해주고 싶은그런 사람이 난 생겼어아직 요리는 잘 못하지만나 연습하고 있어요나 그댈 위해 몰래 감춰놓은애교도 있는 걸매일 지루하지 않게웃게 해줄 텐데너는 내 맘 모르지Ah Choo널 보면 재채기가 나올 것 같아너만 보면 해주고픈 얘기가 참 많아나의 입술이너무 간지러워 참기가 힘들어Ah Choo내 맘에 꽃가루가 떠다니나봐널 위해서 해주고픈 일들이 참 많아나의 마음이 내 사랑이더 이상은 삼키기 힘들어다정하게 깨우고 싶은그런 사람이 난 생겼어아침잠이 좀 많긴 해도잘 일어날 수 있어요그 사람이 막 지쳐 보일 때면내가 더 슬픈 걸혹시 내가 필요할 땐 거기 있어줄게너는 내 맘 모르지Ah Choo널 보면 재채기가 나올 것 같아너만 보면 해주고픈 얘기가 참 많아나의 입술이너무 간지러워 참기가 힘들어Ah Choo내 맘에 꽃가루가 떠다니나봐널 위해서 해주고픈 일들이 참 많아나의 마음이 내 사랑이더 이상은 삼키기 힘들어소중한 너의 친구란 그 말이나는 그 말이 참 싫다밤새 쓰다만 편지와말 하지 못한 내 사랑Ah Choo널 보면 재채기가 나올 것 같아너만 보면 해주고픈 얘기가 참 많아나의 입술이너무 간지러워 참기가 힘들어Ah Choo내 맘에 꽃가루가 떠다니나 봐널 위해서 해주고픈 일들이 참 많아나의 마음이 내 사랑이더 이상은 삼키기 힘들어")

In [ ]:
result[0]['summary_text']

'다정하게 깨우고 싶은그런 사람이 난 생겼어아침잠이 좀 많긴 해도잘 일어날 수 있어요그 사람이 막 지쳐 보일 때면내가 더 슬픈 걸혹시 내가 필요할 땐 거기 있어줄게너는 내 맘 모르지Ah Choo널 보면 재채기가 나올 것 같아너만 보면 해주고픈 얘기가 참 많아나의 입술이너무 간지러워 참기가 힘들어AhChoo내 맘에 꽃가루가 떠다니나 봐널 위해서 해주고픈 일들이 참 많아나와 나의 마음이 내 사랑이더 이상은 삼키기 힘들어'

In [ ]:
song_df['Lyrics']

0       보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1       다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2       해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3       한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4       늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
                              ...                        
2628    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630    지누션함께면 너도          새로 태어난 힙합 빠삐용            지누...
2631    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
2632    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 2633, dtype: object

In [ ]:
len(song_df['Lyrics'])

2633

In [ ]:
lst = []
for i, lyrics in enumerate(tqdm(song_df['Lyrics'])):
    summary = pipe(lyrics)[0]['summary_text']
    lst.append(summary)

    # 텍스트 파일로 저장하기
    if i % 200 == 0:
        file = open(f"lst_{i}.txt", "w")
        file.write(str(lst))
        file.close()

  0%|          | 9/2633 [00:22<1:39:01,  2.26s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
100%|██████████| 2633/2633 [1:37:19<00:00,  2.22s/it]


In [ ]:
file = open("lst_all.txt", "w")
file.write(str(lst))
file.close()

In [ ]:
# 텍스트 파일 불러오기 예제
filename = 'lst_all.txt'
f = open(filename, 'r')     # mode = 부분은 생략해도 됨
txt_str = f.read()
txt_lst = eval(txt_str)

In [ ]:
len(txt_lst)

2633

In [ ]:
song_df

,Song_ID,Genre,Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630,90105,랩/힙합,This is the tihs that y'all ain't ready for Th...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...


In [ ]:
song_df['Summarized Lyrics'] = txt_lst

In [ ]:
song_df

,Song_ID,Genre,Lyrics,Summarized Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...,보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...,이젠 혼자 남겨진 내가낯설고 어색하긴 하지만모두 끝나버린 사랑이었기에너를 떠나 보낼...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...,널 보고싶다고 수만번 불러보면 너도 나를한번쯤 생각하는지 사랑이 올까요 너도 나와 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...,늦은 밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한...
...,...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...,하루가 지나어쩔수 없이 돌아가던 길너는 내게 다가와시간을 멈추고 싶어내맘은 달라 아...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...,헤어지자 난 다시 갈래 여자 그냥 가마음이 다쳐서 숨도 못 쉬겠어심장에 번져서죽을만...
2630,90105,랩/힙합,This is the tihs that y'all ain't ready for Th...,지누션과 함께면 너도 새로 태어난 힙합 빠삐용 아무도 아무도 날 몰라줄 때 빽 있어...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...,길에서 널 만날 수 있다면 그 날처럼 널 보내지 않을 거야 눈물이 멈춘대도너를 지울...


In [ ]:
song_df.to_excel('가사_스테디셀러_요약.xlsx', index=False)

In [2]:
song_df = pd.read_excel('가사_스테디셀러_요약.xlsx')

In [3]:
song_df

,Song_ID,Genre,Lyrics,Summarized Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...,보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...,이젠 혼자 남겨진 내가낯설고 어색하긴 하지만모두 끝나버린 사랑이었기에너를 떠나 보낼...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...,널 보고싶다고 수만번 불러보면 너도 나를한번쯤 생각하는지 사랑이 올까요 너도 나와 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...,늦은 밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한...
...,...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...,하루가 지나어쩔수 없이 돌아가던 길너는 내게 다가와시간을 멈추고 싶어내맘은 달라 아...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...,헤어지자 난 다시 갈래 여자 그냥 가마음이 다쳐서 숨도 못 쉬겠어심장에 번져서죽을만...
2630,90105,랩/힙합,This is the tihs that y'all ain't ready for Th...,지누션과 함께면 너도 새로 태어난 힙합 빠삐용 아무도 아무도 날 몰라줄 때 빽 있어...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...,길에서 널 만날 수 있다면 그 날처럼 널 보내지 않을 거야 눈물이 멈춘대도너를 지울...


In [18]:
song_df.tail(20)

,Song_ID,Genre,Lyrics,Summarized Lyrics
2613,5824433,랩/힙합,oh oh lonely night생각에 잠기네애타는 마음은 눈물만 삼키네슬픔은 커지...,애타는 마음은 눈물만 삼키네슬픔은 커지네 그녀는 모르네 내 마음은바보같이 아직도 그...
2614,7847861,댄스,Dumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Du...,낭만적인 영화를 난 꿈꿔왔지만네 사랑은 내 손에늘 땀을 쥐게 해 마네킹 인형처럼하나...
2615,5709750,"발라드, 국내드라마",사랑은 소리 없이 찾아와내 가슴을 떨리게 만들죠 아무런 이유 없이 전화를 그냥만져 ...,사랑은 소리 없이 찾아와 내 가슴을 떨리게 만들죠 아무 이유 없이 전화를 그냥만져 ...
2616,7847865,댄스,Oh Boy Oh Boy Oh Boy Oh BoyOh Boy 날 비춘 햇빛Oh Bo...,꿈에 그려온 환상 속 그가 날 찾아와찾아와 찾아와또 말없이 다가와 숨 멎는 미소만넌...
2617,491451,성인가요/트로트,그대는 나의 모자람을 미소로 채워주고감싸주는 유일한 사람그대는 나의 마음속에 해가 ...,간 주 중 그대는 나의 모자람을 미소로 채워주고감싸주는 오직 한사람그대는 나의 마음...
2618,4562881,댄스,한번 보면 두 번 더 보고 싶어 두번 세번 보면 너를 더 안고 싶어 너와 커플링 커...,한번 보면 두 번 더 보고 싶어 두번 세번 보면 너를 더 안고 싶어 너와 커플링 커...
2619,1941444,"인디음악, 포크/블루스",나의 사랑하는 남자친구는허리가 좋지 않아서앉아있다 일어 설 때면언제나 에구구구구구구...,나의 사랑하는 남자친구는허리가 좋지 않아서앉아있다 일어 설 때면언제나 에구구구 구구...
2620,4054983,댄스,Ooh Ooh Ooh Ooh Ooh Ooh Ooh Ooh Ooh Ooh 귀엽게Ooh...,귀엽게 예쁘게하루에도 열번씩 네 전화번호 지워보고너를 또 지워보고생각에서 지워봐도너...
2621,57287,성인가요/트로트,희미한 갈색 등불 아래 싸늘히 식어가는 커피잔사람들은 모두가 떠나고 나만홀로 남은 ...,추억속의 그 찻집 우리는 나란히 커피를 마시며 뜨거운 가슴 나누었는데음악에 취해서 ...
2622,1368011,"인디음악, 록/메탈",난 내가 말야 스무살 쯤엔요절할 천재일 줄만 알고어릴 땐 말야 모든 게다 간단하다 ...,스무 살 쯤엔 요절할 천재일 줄만 알고어릴 땐 말야 모든 게다 간단하다 믿었지이제 ...


In [16]:
song_df[song_df['Song_ID'] == 4562881]['Lyrics'].values[0]

"한번 보면 두 번 더 보고 싶어 두번 세번 보면 너를 더 안고 싶어 너와 커플링 커플링 손에 끼고서 함께 이 길을 걷고 싶어 난  매일 매일 봐도 난 더 좋아져 두번 세번 나의 볼을 꼬집어 봐도  마치 Dreaming Dreaming 꿈을 꾸는 듯 생각만 해도 난 미소가  Mr. Chu 입술 위에 Chu 달콤하게 Chu 온몸에 난 힘이 풀려 내 맘 흔들 흔들어 날 흔들어놔요 I'm falling falling for your love  Hey you 입술 위에 Chu Everyday with you 널 보면 내 눈이 감겨 몰래 살짝 다가와 또 키스해줄래 내 꿈결 같은 넌 나만의 Mr. Chu부드러운 감촉 잊을 수 없어 화끈거리는 내 얼굴 빨개지는 걸  It's so lovely lovely 사랑스러워 난 네가 자꾸만 좋아져 Mr. Chu 입술 위에 Chu 달콤하게 Chu 온몸에 난 힘이 풀려 내 맘 흔들 흔들어 날 흔들어놔요 I'm falling falling for your love  Hey you 입술 위에 Chu Everyday with you 널 보면 내 눈이 감겨 몰래 살짝 다가와 또 키스해줄래 내 꿈결 같은 넌 나만의  내 소원을 들어줘요 영원한 사랑 이뤄주길 짜릿짜릿한 느낌절대 맘 변하지 않기평생 나만 바라봐줘 baby Mr. Chu입술 위에 Chu달콤하게 Chu온몸에 난 힘이 풀려내 맘 흔들 흔들어 날 흔들어놔요I'm falling falling for your loveHey you 입술 위에 ChuEveryday with you널 보면 내 눈이 감겨몰래 살짝 다가와 또 키스해줄래내 꿈결 같은 넌 나만의 Mr. Chu"

In [17]:
song_df[song_df['Song_ID'] == 4562881]['Summarized Lyrics'].values[0]

'한번 보면 두 번 더 보고 싶어 두번 세번 보면 너를 더 안고 싶어 너와 커플링 커플링 손에 끼고서 함께 이 길을 걷고 싶어 난 매일 매일 봐도 난 더 좋아져 입술 위에 달콤하게 온몸에 난 힘이 풀려 내 맘 흔들 흔들어 날 흔들어놔요 입술 위에 널 보면 내 눈이 감겨 몰래 살짝 다가와 또 키스해줄래 내 꿈결 같은 넌 나만의 부드러운 감촉 잊을 수 없어'

In [19]:
song_df[song_df['Song_ID'] == 7847861]['Lyrics'].values[0]

"Dumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb Dumb Dumb너 땜에 하루 종일 고민하지만널 어떡해야 좋을지 잘 모르겠어 난 OhBaby baby baby baby baby youPlay me play me play meplay me play me그 눈빛은 날 아찔하고 헷갈리게 해내 이성적인 감각들을 흩어지게 해 OhBaby baby baby baby baby youCrazy crazy crazy crazy crazy마네킹 인형처럼하나부터 열까지 다 어색하지평소같이 하면 되는데또 너만 보면 시작되는 바보 같은 춤눈 코 입 표정도 팔 다리 걸음도내 말을 듣지 않죠Dumb Dumb Dumb DumbDumb Dumb Dumb Dumb심장의 떨림도 날뛰는 기분도맘대로 되질 않죠Dumb Dumb Dumb DumbDumb Dumb Dumb DumbDumb Dumb DumbDumb Dumb DumbDumb Dumb Dumb DumbDumb Dumb DumbDumb Dumb Dumb DumbDumb Dumb Dumb DumbDumb Dumb DumbDumb Dumb DumbDumb Dumb Dumb DumbDumb Dumb DumbDumb Dumb Dumb DumbDumb Dumb Dumb Dumb낭만적인 영화를 난 꿈꿔왔지만네 사랑은 내 손에늘 땀을 쥐게 해 OhBaby baby baby baby babyYou make mecrazy crazy crazy crazy hey마네킹 인형처럼하나부터 열까지 다 어색하지평소같이 하면 되는데또 너만 보면 시작되는 바보 같은 춤Dumb Dumb Dumb Dumb Dumb DumbDum

In [20]:
song_df[song_df['Song_ID'] == 7847861]['Summarized Lyrics'].values[0]

'낭만적인 영화를 난 꿈꿔왔지만네 사랑은 내 손에늘 땀을 쥐게 해 마네킹 인형처럼하나부터 열까지 다 어색하지평소같이 하면 되는데또 너만 보면 시작되는 바보 같은 춤눈 코 입 표정도 팔 다리 걸음도내 말을 듣지 않죠 심장의 떨림도 날뛰는 기분도맘대로 되질 않죠'